# Ratings_cleaning musi zostać uruchomiony jako pierwszy
Notebook `Ratings_cleaning.ipynb` usuwa niektóre z wystawionych ocen, co zmniejsza ich liczbę u użytkowników. Następnie usuwa użytkowników, którzy mają mniej niż 20 ocen i zapisuje nowy zbiór danych do pliku `user_features_2.csv`.

In [20]:
import pandas as pd

In [21]:
df = pd.read_csv('../datasets/user_features_2.csv')

In [22]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200846 entries, 0 to 200845
Data columns (total 25 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             200846 non-null  int64  
 1   userId                 200846 non-null  int64  
 2   num_rating             200846 non-null  int64  
 3   avg_rating             200846 non-null  float64
 4   weekend_watcher        200846 non-null  int64  
 5   type_of_viewer         200846 non-null  object 
 6   genre_Action           199613 non-null  float64
 7   genre_Adventure        199878 non-null  float64
 8   genre_Animation        173940 non-null  float64
 9   genre_Comedy           200378 non-null  float64
 10  genre_Crime            198682 non-null  float64
 11  genre_Documentary      72068 non-null   float64
 12  genre_Drama            200781 non-null  float64
 13  genre_Family           188451 non-null  float64
 14  genre_Fantasy          196283 non-nu

# Uzupełnienie brakujących ocen 

In [23]:
genre_columns = ['genre_Action','genre_Adventure','genre_Animation','genre_Comedy','genre_Crime','genre_Documentary','genre_Drama','genre_Family','genre_Fantasy','genre_History','genre_Horror','genre_Music','genre_Mystery','genre_Romance','genre_Science Fiction','genre_TV Movie','genre_Thriller','genre_War','genre_Western']

for col in genre_columns:
    df[col] = df[col].fillna(df['avg_rating'])

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200846 entries, 0 to 200845
Data columns (total 25 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             200846 non-null  int64  
 1   userId                 200846 non-null  int64  
 2   num_rating             200846 non-null  int64  
 3   avg_rating             200846 non-null  float64
 4   weekend_watcher        200846 non-null  int64  
 5   type_of_viewer         200846 non-null  object 
 6   genre_Action           200846 non-null  float64
 7   genre_Adventure        200846 non-null  float64
 8   genre_Animation        200846 non-null  float64
 9   genre_Comedy           200846 non-null  float64
 10  genre_Crime            200846 non-null  float64
 11  genre_Documentary      200846 non-null  float64
 12  genre_Drama            200846 non-null  float64
 13  genre_Family           200846 non-null  float64
 14  genre_Fantasy          200846 non-nu

In [24]:
df.to_hdf('../datasets/user_features_clean_zlib.h5', key='df', mode='w', complevel=9, complib='zlib')